In [1]:
import logging
import sys

logging.basicConfig(level=logging.DEBUG, stream=sys.stdout)

In [2]:
%%time
from bedarev_syntax import search, Text, Post, GrammeRequest

with open("./data/clear_text.txt", 'r', encoding='utf8') as fp:
    TEXTS = [Text.parse(line) for line in fp.read().split('\n')]

INFO:pymorphy2.opencorpora_dict.wrapper:Loading dictionaries from D:\ProgramData\Anaconda3\lib\site-packages\pymorphy2_dicts\data
INFO:pymorphy2.opencorpora_dict.wrapper:format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168
Wall time: 1min 54s


In [3]:
from typing import Sequence, Union, Tuple, Optional

def print_search(request: Sequence[Union[str, GrammeRequest]],
                         texts: Sequence[Text], window: int = 15):
    for ind, text in enumerate(texts):
        poss = search(request, text)
        if poss:
            print(f'{text.raw[max(0, poss[0]-window):poss[1]+window]}')


In [4]:
print_search(['и это', Post.VERB.request()], TEXTS)

ошо совпадали, и это давало их коллективам
лжен работать, и это будет гораздо лучшим
чительно выше, и это затрудняет взаимовыгодную
чительно выше, и это затрудняет взаимовыгодную
 они работают, и это считается нормой (Сивак,


In [5]:
print_search([Post.VERB.request(), Post.VERB.request()], TEXTS)

ельством этого времени был приход в колле
 катушки Рассм атри вается работа магнитн
ого равновесия меняются при изменении не т
ассмотренным в работах–.Здесь представляет интерес исслед
разных сторон. Заметим при этом, что факт
Петерсон привёл пример постановки для
Модель Тоута и Прингла даёт следующее выра
 этой ситуации стали применятся новые математи
риваемых задач остается открытым.При повышении поря
е формы записи оказывается при схемной реализ
 необратимость времени понимается несколько по-р
ые силы уже не работают при таких масштаба
Такая геометриясопла была выбрана на осн
утем аккреции, мог вырасти за 500–800 млн
ки оптимальным будет полет к Земле по «об
шие результаты получаются при величинах в не
ем времени они продолжают расти и медленнопере
3и температуре используются при вычислении раз
деталей, и шум почти исчезает.
 распределение порождает при подстановке в 
едством данной 3Dмодели рассматривалось подобное взаим
 до настоящего времени отсутствует.
ередних стоп к берегу ус

производная по времени понимается в смысле Капут
я экономика не стала расти сразу по неско
ОС) на 2011 г. работало почти в 5 раз меньше
Тэшфела рассматривается как та часть и
няемая в коде, имеет начало в принципе адд
перкомпьютеров эточисло может быть увеличено
шем промежутке времени требуется рассмотрение о
по отношению к рублю был рассчитан на о
я форсайта уже используется при анализе и прог
раструктуры, и порождает три ключевых инфра
темах процессы происходят при наличии азимут
машиностроения сократилась почти вдвое [10, С. 
липания частиц пыли являются: ван-дер-вааль
 курсу, Россия занимает 80-е место среди 15
На сегодняшний день существуют различные подх
тября 2006 г.) выросли почти в 4 раза, капи
2007)Выработка мер проходила в условиях ост
дкости на поле учитывается при записизакона О
окорректора от времени при разныхh.
х асимптотик в окрестноти точек푧1,2с асимптотиками(
 гистерезисной петли Рассмотрим гистерезисные 
апример, ЦБ РФ опубликовал три сценария: базо
Соционет допуска

ического блока Раздел содержит описание алгор
тся отражённые боли и/или изменения боле
акое положение дел наблюдается, например, в п
ения векторных полей является группа многоча
та инвестиций, был опровергнут опытом многих 
фазной границы приводит при усреднении к п
магнитные поля возникает при решении широко
ассматриваемой 3Dмодели обнаруживается некоторое отно
резки частично компенсируются при реконструкции,
х. 7.7. Теперь рассмотрим пример распада разрыв
 1-го порядка. Рассматривался случай одномерного ур
зультативности учитываются при увольнении, пр
ля того, чтобы правила стали институтами, н
зультативности учитываются при увольнении, пр
ров-рыночников начала 90-х годов: необход
я деятельность ФСРЖС будет осуществляться
ек и верёвочек осуществляют весь процесс.
ых вычислений, довершают дело. Не зная уже з
6]. В период с начала 1980-х гг. до 2011 г.
зультативности учитываются при увольнении, пр
 второй группы используют при оценке промежу
 службы только начинает исследоваться (см.)